In [1]:
import numpy as np
import sympy as sp
from sympy import *
from sympy import Matrix
import matplotlib.pyplot as pyplot
import matplotlib.gridspec as gridspec
import matplotlib.cm as cm
from numpy import linalg
import cmath
#%matplotlib inline
from mpl_toolkits import mplot3d
import time
#import numba
#from numba import jit

In [2]:
def U_calc(v1,v2):
    U = np.vdot(v1,v2)
    return U/abs(U)


##### CALCULATION OF BRAKETS #####
def braket(v1,observable,v2):
    return np.vdot(v1,np.dot(observable,v2))


##### FUKUI CURVATURE FOR DIRAC SYSTEM #####
def F(H,n,kx,ky,dkx,dky,DP_x,DP_y,Delta): # algorithm based on the FHS method for a single plaquette
    ''' Inputs: Hamiltonian of the system under considerations (function of kx,ky),
                Coordinates of a point in the Brillouin Zone (BZ),
                Dimensions of one plaquette of the BZ.
        Outputs: Berry curvature at point (kx,ky) in the BZ'''
    
    eigV_0 = linalg.eigh(H(kx,ky,DP_x,DP_y,Delta))[1][:,n]
    eigV_1 = linalg.eigh(H(kx+dkx,ky,DP_x,DP_y,Delta))[1][:,n]
    eigV_2 = linalg.eigh(H(kx+dkx,ky+dky,DP_x,DP_y,Delta))[1][:,n]
    eigV_3 = linalg.eigh(H(kx,ky+dky,DP_x,DP_y,Delta))[1][:,n]
    
    U_01 = U_calc(eigV_0,eigV_1)
    U_12 = U_calc(eigV_1,eigV_2)
    U_23 = U_calc(eigV_2,eigV_3)
    U_30 = U_calc(eigV_3,eigV_0)
    
    return (np.log(U_01*U_12*U_23*U_30)/dkx/dky).imag


##### DIRAC HAMILTONIAN #####

def H_Dirac(kx,ky,DP_x,DP_y,Delta): # construct the Dirac Hamiltonian around Dirac cone
    
    vF = 3/2
    
    sigma_x = np.array([[0,1],[1,0]])
    sigma_y = np.array([[0,-1j],[1j,0]])
    sigma_z = np.array([[1,0],[0,-1]])
            
    return vF*((kx-DP_x)*sigma_x+(ky-DP_y)*sigma_y)+Delta*sigma_z

##### ANALYTICAL FORM OF HONEYCOMB SPECTRUM #####

def E(kx,ky,Delta,t,delta_list):
    temporary = 0*1j
    k = np.array([kx,ky])
    for i in range(len(t)):
        temporary += t[i]*np.exp(-1j*np.dot(k,delta_list[i])) 
    
    return np.sqrt(Delta**2+abs(temporary)**2)

##### ANALYTICAL FORM OF HONEYCOMB SPECTRUM NEAR DIRAC POINT #####

def E_DP(kx,ky,DP_x,DP_y,Delta):
    vF = 1.5
    return np.sqrt(vF**2*(kx-DP_x)**2+vF**2*(ky-DP_y)**2+Delta**2)

##### HONEYCOMB HAMILTONIAN #####

def H_honeycomb(kx,ky,DP_x,DP_y,Delta):
    
    H_honey = 1j*np.zeros((2,2))
    a = 1
    t1 = 1
    t2 = 1
    t3 = 1
    t = np.array([t1,t2,t3])
    d1 = a/2*np.array([1,np.sqrt(3)])
    d2 = a/2*np.array([1,-np.sqrt(3)])
    d3 = a*np.array([-1,0])
    d = [d1,d2,d3]
    k = np.array([kx,ky])
    
    H_honey[0,0] = Delta 
    H_honey[1,1] = -Delta
    
    for i in range(len(t)):
        H_honey[1,0] += t[i]*np.exp(1j*np.dot(k,d[i])) 
        H_honey[0,1] += t[i]*np.exp(-1j*np.dot(k,d[i])) 
        
    return H_honey


##### ANALYTICAL CURVATURE OF HONEYCOMB NEAR DIRAC POINT #####

def Omega(kx,ky,DP_x,DP_y,vF,Delta):
    return 1/2*vF**2*Delta/np.sqrt(vF**2*(kx-DP_x)**2+vF**2*(ky-DP_y)**2+Delta**2)**3

##### ANALYTICAL DERIVATIVE OF HONEYCOMB HAMILTONIAN IN FCT OF Kx,ky #####

def dH_d(kx,ky): # for Honeycomb hamiltonian
    
    dH_dkx = 1j*np.zeros((2,2))
    dH_dky = 1j*np.zeros((2,2))
    a = 1
    t1 = 1
    t2 = 1
    t3 = 1
    t = np.array([t1,t2,t3])
    d1 = a/2*np.array([1,np.sqrt(3)])
    d2 = a/2*np.array([1,-np.sqrt(3)])
    d3 = a*np.array([-1,0])
    d = np.array([d1,d2,d3])
    k = np.array([kx,ky])
     
    for i in range(len(t)):
        dH_dkx[1,0] += 1j*t[i]*d[i,0]*np.exp(1j*np.dot(k,d[i])) 
        dH_dkx[0,1] += -1j*t[i]*d[i,0]*np.exp(-1j*np.dot(k,d[i]))
        
    for i in range(len(t)):
        dH_dky[1,0] += 1j*t[i]*d[i,1]*np.exp(1j*np.dot(k,d[i])) 
        dH_dky[0,1] += -1j*t[i]*d[i,1]*np.exp(-1j*np.dot(k,d[i]))
    
    return dH_dkx,dH_dky

##### NIU ALGORITHM TO FIND BERRY CURVATURE OF HONEYCOMB #####

def F_Niu(H,n,kx,ky,DP_x,DP_y,Delta): # calculates the Berry curv in kx,ky by Niu method
    
    F_Niu_n = 0 
    eigValues = linalg.eigh(H(kx,ky,DP_x,DP_y,Delta))[0]
    eigVectors = linalg.eigh(H(kx,ky,DP_x,DP_y,Delta))[1]
    En = linalg.eigh(H(kx,ky,DP_x,DP_y,Delta))[0][n]
    eigVectors_n = linalg.eigh(H(kx,ky,DP_x,DP_y,Delta))[1][:,n]
    
    deriv_H = dH_d(kx,ky)
    dH_dkx = deriv_H[0]
    dH_dky = deriv_H[1]
    
    for n_prime in range(len(eigVectors)):
        if (n_prime!=n):
            eigVectors_n_prime = eigVectors[:,n_prime]
            En_prime = eigValues[n_prime]
            
            temporary1 = braket(eigVectors_n,dH_dkx,eigVectors_n_prime)
            #print(temporary1)
            temporary2 = braket(eigVectors_n_prime,dH_dky,eigVectors_n)
            #print(temporary2)
            #print(kx)
            F_Niu_n +=  (temporary1*temporary2-np.conj(temporary1*temporary2))/(En-En_prime)**2
            #print(F_Niu_n)
            
    return F_Niu_n